This notebook is the part of the series **Mastering Data Structures for Databases** which you can find here at https://medium.com/@utkarshpriyadarshi5026/list/mastering-data-structures-for-databases-0f3f74559d7e

The article about the skiplist can be found here: https://medium.com/@utkarshpriyadarshi5026/mastering-data-structures-for-databases-part-1-skiplists-e66f0831ac0b 

# Skip Lists: A Comprehensive Guide

## Introduction

Skip Lists are a probabilistic data structure that provide a way to store a sorted list of elements while allowing fast search, insertion, and deletion operations. They were introduced by William Pugh in 1989 as an alternative to balanced trees and other complex data structures.

## How Skip Lists Work

A Skip List can be thought of as a hierarchy of sorted linked lists. The bottom level is a regular sorted linked list containing all elements. Each higher level acts as an "express lane" for the lists below, allowing for faster traversal.

### Key Concepts:

1. **Levels**: Each element in a Skip List can appear in multiple levels. The number of levels for each element is randomly determined when the element is inserted.

2. **Probabilistic Balance**: The random level selection ensures that, on average, the Skip List maintains a balanced structure without the need for explicit rebalancing operations.

3. **Express Lanes**: Higher levels in the Skip List act as shortcuts, allowing the search operation to skip many elements and quickly narrow down the search range.

## Basic Operations

### 1. Search

- Start from the highest level of the first (header) node.
- Move forward while the next node's key is less than the target key.
- If can't move forward, move down a level.
- Repeat until the target key is found or determined to be absent.

**Time Complexity**: O(log n) on average

### 2. Insert

- Perform a search to find the position for insertion.
- Randomly determine the number of levels for the new node.
- Update the necessary links at each level.

**Time Complexity**: O(log n) on average

### 3. Delete

- Perform a search to find the node to be deleted.
- Update the links to remove references to the deleted node.
- Adjust the list's maximum level if necessary.

**Time Complexity**: O(log n) on average


## Advantages of Skip Lists

1. **Simplicity**: Skip Lists are relatively simple to implement compared to balanced trees.
2. **Efficient Operations**: Provide O(log n) average time complexity for search, insert, and delete operations.
3. **Probabilistic Balance**: No need for complex rebalancing operations.
4. **Memory Efficiency**: Can be more space-efficient than some tree structures.

## Disadvantages of Skip Lists

1. **Probabilistic Nature**: Performance guarantees are average-case, not worst-case.
2. **Extra Memory**: Requires extra memory for the express lanes (higher levels).
3. **Not Cache-Friendly**: Non-contiguous memory allocation can lead to cache misses.

## Applications

- Database Systems: Used in some database index implementations.
- In-Memory Databases: Efficient for maintaining sorted data in memory.
- File Systems: Can be used for managing file metadata.
- Cache Systems: Efficient for implementing LRU caches.

## Conclusion

Skip Lists offer a unique balance of simplicity and efficiency, making them a valuable tool in a programmer's toolkit. While they may not always outperform balanced trees, their ease of implementation and good average-case performance make them an attractive choice for many applications, especially where simplicity is valued or where the probabilistic nature of their performance is acceptable.

In [2]:
import random

class Node:
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.forward = []  # List of references to nodes at different levels

class SkipList:
    def __init__(self, max_level=16, p=0.5):
        self.max_level = max_level
        self.p = p
        self.header = Node(None, None)
        self.header.forward = [None] * max_level
        self.level = 0

    def random_level(self):
        """Generate a random level for a new node."""
        level = 0
        while random.random() < self.p and level < self.max_level - 1:
            level += 1
        return level

    def insert(self, key, value):
        """Insert a key-value pair into the Skip List."""
        update = [None] * self.max_level
        current = self.header

        # Find the position to insert the new node
        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
            update[i] = current

        current = current.forward[0]

        if current is None or current.key != key:
            # Generate a random level for the new node
            new_level = self.random_level()

            # If the new level is higher than the current skip list level,
            # update the header's forward references
            if new_level > self.level:
                for i in range(self.level + 1, new_level + 1):
                    update[i] = self.header
                self.level = new_level

            new_node = Node(key, value)

            # Update the forward references for the new node and its predecessors
            for i in range(new_level + 1):
                new_node.forward.append(update[i].forward[i])
                update[i].forward[i] = new_node
        else:
            # If the key already exists, update its value
            current.value = value

    def search(self, key):
        """Search for a key in the Skip List."""
        current = self.header

        # Start from the highest level and move down
        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]

        current = current.forward[0]

        if current and current.key == key:
            return current.value
        return None

    def delete(self, key):
        """Delete a key from the Skip List."""
        update = [None] * self.max_level
        current = self.header

        # Find the node to delete and keep track of its predecessors
        for i in range(self.level, -1, -1):
            while current.forward[i] and current.forward[i].key < key:
                current = current.forward[i]
            update[i] = current

        current = current.forward[0]

        if current and current.key == key:
            # Update the forward references to skip the node being deleted
            for i in range(len(current.forward)):
                if update[i].forward[i] != current:
                    break
                update[i].forward[i] = current.forward[i]

            # Update the skip list level if necessary
            while self.level > 0 and self.header.forward[self.level] is None:
                self.level -= 1

    def display(self):
        """Display the Skip List structure."""
        for level in range(self.level, -1, -1):
            print(f"Level {level}: ", end="")
            node = self.header.forward[level]
            while node:
                print(f"({node.key}: {node.value}) ", end="")
                node = node.forward[level]
            print()





In [3]:
sl = SkipList()

# Insert some key-value pairs
sl.insert(3, "three")
sl.insert(6, "six")
sl.insert(7, "seven")
sl.insert(9, "nine")
sl.insert(12, "twelve")
sl.insert(19, "nineteen")
sl.insert(17, "seventeen")
sl.insert(26, "twenty-six")
sl.insert(21, "twenty-one")
sl.insert(25, "twenty-five")

print("Skip List after insertions:")
sl.display()

# Search for some keys
print("\nSearching for keys:")
print("Key 19:", sl.search(19))
print("Key 20:", sl.search(20))

# Delete some keys
sl.delete(19)
sl.delete(25)

print("\nSkip List after deletions:")
sl.display()

Skip List after insertions:
Level 6: (17: seventeen) 
Level 5: (17: seventeen) 
Level 4: (17: seventeen) 
Level 3: (9: nine) (17: seventeen) 
Level 2: (9: nine) (17: seventeen) (21: twenty-one) 
Level 1: (9: nine) (12: twelve) (17: seventeen) (21: twenty-one) 
Level 0: (3: three) (6: six) (7: seven) (9: nine) (12: twelve) (17: seventeen) (19: nineteen) (21: twenty-one) (25: twenty-five) (26: twenty-six) 

Searching for keys:
Key 19: nineteen
Key 20: None

Skip List after deletions:
Level 6: (17: seventeen) 
Level 5: (17: seventeen) 
Level 4: (17: seventeen) 
Level 3: (9: nine) (17: seventeen) 
Level 2: (9: nine) (17: seventeen) (21: twenty-one) 
Level 1: (9: nine) (12: twelve) (17: seventeen) (21: twenty-one) 
Level 0: (3: three) (6: six) (7: seven) (9: nine) (12: twelve) (17: seventeen) (21: twenty-one) (26: twenty-six) 
